# Spotify - Search by Speech Keywords



Q1 as high valence high arousal (HVHA) - Happy,

Q2 as low valence high arousal (LVHA) - Angry,

Q3 as low valence low arousal (LVLA) - Sad,

Q4 as high valence low arousal (HVLA) - Calm.


In [ ]:
!pip install spotipy

In [ ]:
from google.colab import userdata

In [ ]:
import json, requests
import numpy as np

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
sp = spotipy.Spotify(
    auth_manager=SpotifyClientCredentials(
        client_id='EBMP_APP_CLIENT_ID',
        client_secret='EBMP_CLIENT_SECRET'
        )
    )

In [ ]:
output = sp.search("Cold Nights", limit=30, offset=0, type='track', market='US')

In [ ]:
class EmotionCalculator:
    @staticmethod
    def calculate_angle(arousal, valence):
        # Scale arousal and valence to [-1, 1] range
        arousal_scaled = 2 * arousal - 1
        valence_scaled = 2 * valence - 1
        # Calculate the angle in radians
        angle_rad = np.arctan2(arousal_scaled, valence_scaled)
        # Convert radians to degrees
        angle_deg = np.degrees(angle_rad)
        # Ensure angle is positive
        if angle_deg < 0:
            angle_deg += 360
        return angle_deg

    @staticmethod
    def determine_quadrant(angle_deg):
        if 0 <= angle_deg < 90:
            return "QUAD1"
        elif 90 <= angle_deg < 180:
            return "QUAD2"
        elif 180 <= angle_deg < 270:
            return "QUAD3"
        else:
            return "QUAD4"

    @staticmethod
    def determine_emotion(quadrant_name):
        if quadrant_name == "QUAD1":
            return "Happiness"
        elif quadrant_name == "QUAD2":
            return "Anger"
        elif quadrant_name == "QUAD3":
            return "Sadness"
        else:
            return "Calmness"

In [ ]:
available_markets = ["US", "GB", "DE", "FR", "BR", "JP", "RU", "IN", "KR", "ES"]

def get_preview_url(track_id):
    for market in available_markets:
        try:
            output = sp.track(track_id, market=market)
            if output['preview_url']:
                return output['preview_url']
        except Exception as e:
            print(f"Error occurred: {e}")
    return "No preview url found"

In [ ]:
# Number of limits specified
limit = 20

# List to store track information
track_info_list = []

for i in range(min(limit, len(output["tracks"]["items"]))):
    track_info = {}
    track = output["tracks"]["items"][i]

    # Extracting track ID and name
    track_info['id'] = track['id']
    track_info['name'] = track['name']

    # Extracting artists' names
    track_info['artists'] = [artist['name'] for artist in track['artists']]

    # Extracting images details
    track_info['images'] = track['album']['images']

    # Extracting preview URL if available
    if 'preview_url' in track and track['preview_url'] is not None:
      track_info['preview_url'] = track['preview_url']
    else:
      track_info['preview_url'] = get_preview_url(track['id'])

    # Extract the track features
    track_features = sp.audio_features(track['id'])
    track_info['arousal'] = track_features[0]['energy']
    track_info['danceability'] = track_features[0]['danceability']
    track_info['acousticness'] = track_features[0]['acousticness']
    track_info['valence'] = track_features[0]['valence']

    # Extract track URL
    track_info['track_url'] = track['external_urls']['spotify']

    # Check if all fields are filled and preview_url is not "[No preview url found]"
    if all(track_info.values()) and track_info['preview_url'] != "No preview url found":
        # Calculate angle
        track_info['angle_deg'] = EmotionCalculator.calculate_angle(arousal=track_info['arousal'], valence=track_info['valence'])

        # Determine quadrant
        track_info['quadrant'] = EmotionCalculator.determine_quadrant(angle_deg=track_info['angle_deg'])

        # Determine emotion
        track_info['emotion'] = EmotionCalculator.determine_emotion(quadrant_name=track_info['quadrant'])

        # Append track information to the list
        track_info_list.append(track_info)

In [ ]:
track_info_list

[{'id': '1bwlBQYp4KgABzsi9s7nzx',
  'name': 'Cold Nights',
  'artists': ['Hulvey'],
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b2733aaaeab5117e8b8f776596cb',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e023aaaeab5117e8b8f776596cb',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d000048513aaaeab5117e8b8f776596cb',
    'width': 64}],
  'preview_url': 'https://p.scdn.co/mp3-preview/f29afc5cb3caecd897937b4f9a71dd7fe4b6e603?cid=7a5ccd9e742a4adc9e73041d836087be',
  'arousal': 0.571,
  'danceability': 0.762,
  'acousticness': 0.0342,
  'valence': 0.395,
  'track_url': 'https://open.spotify.com/track/1bwlBQYp4KgABzsi9s7nzx',
  'angle_deg': 145.93381678575582,
  'quadrant': 'QUAD2',
  'emotion': 'Anger'},
 {'id': '4gKYW7JKLAO5KPwadZASPm',
  'name': 'Cold Nights in Hollywood',
  'artists': ['Buppy.'],
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b273dc719d

In [ ]:
i = 0

for track_info in track_info_list:
      # Increment the track number
      i += 1

      # Printing track information
      print(f"\nTrack {i}:")
      print("Track ID:", track_info['id'])
      print("Track Name:", track_info['name'])

      # Printing artists
      print("Artists:", ', '.join(track_info['artists']))

      # print track url
      print("Track URL:", track_info['track_url'])

      # Printing preview URL
      print("Preview URL:", track_info['preview_url'])

      # Printing Track Features
      print("Arousal:", track_info['arousal'])
      print("Valence:", track_info['valence'])

      # Printing Track Quadrants
      print("Angle:", track_info['angle_deg'])
      print("Quadrant:", track_info['quadrant'])
      print("Emotion:", track_info['emotion'])


Track 1:
Track ID: 1bwlBQYp4KgABzsi9s7nzx
Track Name: Cold Nights
Artists: Hulvey
Track URL: https://open.spotify.com/track/1bwlBQYp4KgABzsi9s7nzx
Preview URL: https://p.scdn.co/mp3-preview/f29afc5cb3caecd897937b4f9a71dd7fe4b6e603?cid=7a5ccd9e742a4adc9e73041d836087be
Arousal: 0.571
Valence: 0.395
Angle: 145.93381678575582
Quadrant: QUAD2
Emotion: Anger

Track 2:
Track ID: 4gKYW7JKLAO5KPwadZASPm
Track Name: Cold Nights in Hollywood
Artists: Buppy.
Track URL: https://open.spotify.com/track/4gKYW7JKLAO5KPwadZASPm
Preview URL: https://p.scdn.co/mp3-preview/05033b2d9f5ecfcfc90b658d35ea878efa6680bd?cid=7a5ccd9e742a4adc9e73041d836087be
Arousal: 0.494
Valence: 0.239
Angle: 181.31691240579326
Quadrant: QUAD3
Emotion: Sadness

Track 3:
Track ID: 7v0mCN6KkwIuDSZJm3Tzl8
Track Name: Cold Nights
Artists: $tupid Young
Track URL: https://open.spotify.com/track/7v0mCN6KkwIuDSZJm3Tzl8
Preview URL: https://p.scdn.co/mp3-preview/1d6b48d0f954a2641960045d3556da27d9503554?cid=7a5ccd9e742a4adc9e73041d836087b

- Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.

- Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.

1. If you're interested in the physical aspect of arousal related to movement and rhythm, danceability might be more suitable.
2. If you're interested in a broader sense of arousal including emotional intensity, energy could be more appropriate.

In [ ]:
your_predicted_emotion = "Anger"

In [ ]:
# Filter tracks by predicted emotion
filtered_tracks = [track for track in track_info_list if track['emotion'] == your_predicted_emotion]

# If the length of filtered_tracks is less than 5, include tracks with other emotions
if len(filtered_tracks) < 5:
  other_tracks = [track for track in track_info_list if track['emotion'] != your_predicted_emotion]
  filtered_tracks += other_tracks[:5 - len(filtered_tracks)]

# Sort tracks by their angle_deg
filtered_tracks.sort(key=lambda track: track['angle_deg'])

# Calculate the mean of angles in filtered tracks using NumPy
mean_angle = np.mean([track['angle_deg'] for track in filtered_tracks])

# Sort tracks by their angle difference from the mean angle using NumPy
sorted_indices = np.argsort(np.abs([track['angle_deg'] - mean_angle for track in filtered_tracks]))

# Get the top 5 tracks with angles closest to the mean angle
top_5_tracks = [filtered_tracks[i] for i in sorted_indices[:5]]

top_5_tracks

[{'id': '1SHSadkt2u93xvES8P2o66',
  'name': 'Cold Nights',
  'artists': ['Pimp Tobi'],
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b2734bdaa267aab9584fb330d4a7',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e024bdaa267aab9584fb330d4a7',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d000048514bdaa267aab9584fb330d4a7',
    'width': 64}],
  'preview_url': 'https://p.scdn.co/mp3-preview/3cf9d4e2cd3bb01aa84362cfb6338f204b3fa67e?cid=7a5ccd9e742a4adc9e73041d836087be',
  'arousal': 0.574,
  'danceability': 0.855,
  'acousticness': 0.0617,
  'valence': 0.385,
  'track_url': 'https://open.spotify.com/track/1SHSadkt2u93xvES8P2o66',
  'angle_deg': 147.23959983693186,
  'quadrant': 'QUAD2',
  'emotion': 'Anger'},
 {'id': '1bwlBQYp4KgABzsi9s7nzx',
  'name': 'Cold Nights',
  'artists': ['Hulvey'],
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b2733aaaeab5117e8b8f

# DF Visualization

In [ ]:
import pandas as pd

df = pd.DataFrame(top_5_tracks)

In [ ]:
df.drop(columns=['images'], inplace=True)

In [ ]:
df

,id,name,artists,preview_url,arousal,danceability,acousticness,valence,track_url,angle_deg,quadrant,emotion
0,1SHSadkt2u93xvES8P2o66,Cold Nights,[Pimp Tobi],https://p.scdn.co/mp3-preview/3cf9d4e2cd3bb01a...,0.574,0.855,0.06170,0.3850,https://open.spotify.com/track/1SHSadkt2u93xvE...,147.239600,QUAD2,Anger
1,1bwlBQYp4KgABzsi9s7nzx,Cold Nights,[Hulvey],https://p.scdn.co/mp3-preview/f29afc5cb3caecd8...,0.571,0.762,0.03420,0.3950,https://open.spotify.com/track/1bwlBQYp4KgABzs...,145.933817,QUAD2,Anger
2,3OxH1yJ35RIN47bUEx3JaX,Cold Nights,[Pauli Gabrieli],https://p.scdn.co/mp3-preview/c887a88a86effcfe...,0.667,0.837,0.02110,0.1150,https://open.spotify.com/track/3OxH1yJ35RIN47b...,156.550428,QUAD2,Anger
3,2Ybfp2mISs0UdJ5qA8Fjvz,Cold Nights,"[Frontières, Hollow Front]",https://p.scdn.co/mp3-preview/f4109c52d2c0346f...,0.991,0.468,0.00646,0.0545,https://open.spotify.com/track/2Ybfp2mISs0UdJ5...,132.218463,QUAD2,Anger
4,7v0mCN6KkwIuDSZJm3Tzl8,Cold Nights,[$tupid Young],https://p.scdn.co/mp3-preview/1d6b48d0f954a264...,0.535,0.774,0.38400,0.2700,https://open.spotify.com/track/7v0mCN6KkwIuDSZ...,171.347458,QUAD2,Anger
